## Librerias

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
%matplotlib inline

Using TensorFlow backend.


In [3]:
import tensorflow

In [4]:
import keras

In [5]:
data = pd.read_csv('../data/data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [6]:
data.shape

(1000, 28)

In [ ]:
attributes = ['mfcc%d'%i for i in range(1,21)]
pd.plotting.scatter_matrix(data[attributes], figsize=(12,8));


In [ ]:
attributes2 = ['chroma_stft','rmse','spectral_centroid', 'spectral_bandwidth', 'rolloff','zero_crossing_rate']
pd.plotting.scatter_matrix(data[attributes2], figsize=(12,8));

## PREPROCESSING



In [7]:
from sklearn import preprocessing


In [8]:
labels = data.iloc[:, -1]
encoder = preprocessing.LabelEncoder()


In [9]:
y = encoder.fit_transform(labels)#se obtienen los labels 
print(y.shape,np.unique(y))

(1000,) [0 1 2 3 4 5 6 7 8 9]


In [10]:
# Dropping de columnas inncesarias
data = data.drop(['filename'],axis=1)

In [11]:
scaler = preprocessing.StandardScaler()#Se estandarizan las caracteristicas Z=(x-u)/s
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [22]:
X[0:10][:]

array([[-3.51748622e-01, -1.07229841e-02, -5.83303337e-01,
        -4.56037034e-01, -4.86166057e-01, -4.91966558e-01,
         3.08252406e-01,  7.02673975e-01, -4.72089912e-01,
         3.63652865e-01, -4.28097656e-01,  3.36698995e-01,
        -8.62289886e-01,  4.99553068e-01, -6.38827279e-01,
         4.09659355e-01, -3.37930649e-01,  6.46275693e-01,
         1.83186590e-01,  7.92921116e-01, -2.65289349e-01,
        -8.67504224e-02,  5.00287206e-01, -2.37191577e-01,
         7.61145087e-03,  6.03498128e-01],
       [-4.61465781e-01, -5.33266150e-01, -9.39066284e-01,
        -3.87667644e-01, -6.49868713e-01, -1.13821966e+00,
        -6.29608118e-01,  7.80889484e-01,  8.23297118e-01,
        -2.51147670e-02,  3.32557120e-01,  5.82147213e-01,
        -3.44690968e-01,  1.26579763e+00, -3.75344267e-01,
         5.26025865e-01,  6.75997471e-02,  1.34823435e-01,
         4.14750611e-01,  4.86403784e-01, -4.39656016e-01,
        -4.62939579e-02,  7.14737546e-01, -5.51897790e-02,
         5.43

In [20]:
#guardamos los valores de las medias y desviaciones estándar
np.save("medias", scaler.mean_)
np.save("desvest", scaler.scale_)

In [14]:
data.shape

(1000, 27)

In [15]:
print(X.shape,y.shape)

(1000, 26) (1000,)


In [16]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## USING KERAS

In [16]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 1s 1ms/step - loss: 2.2089 - acc: 0.2137
Epoch 2/20
800/800 [==============================] - 0s 25us/step - loss: 1.8799 - acc: 0.3337
Epoch 3/20
800/800 [==============================] - 0s 26us/step - loss: 1.6664 - acc: 0.3663
Epoch 4/20
800/800 [==============================] - 0s 26us/step - loss: 1.4917 - acc: 0.4688
Epoch 5/20
800/800 [==============================] - 0s 20us/step - loss: 1.3483 - acc: 0.5487
Epoch 6/20
800/800 [==============================] - 0s 24us/step - loss: 1.2340 - acc: 0.5875
Epoch 7/20
800/800 [==============================] - 0s 22us/step - loss: 1.1226 - acc: 0.6337
Epoch 8/20
800/800 [==============================] - 0s 20us/step - loss: 1.0472 - acc: 0.6575
Epoch 9/20
800/800 [==============================] - 0s 22us/step - loss: 0.9793 - acc: 0.6763
Epoch 10/20
800/800 [==============================] - 0s 19us/step - loss: 0.9167 - acc:

In [36]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 20us/step


In [37]:
print('test_acc: ',test_acc)

test_acc:  0.675


In [25]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    # serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [38]:
"""

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

"""

'\n# load json and create model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n'

In [28]:
"""

#compiling the loaded model
loaded_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
"""

In [34]:
"""

#evaluating the loaded model
test_loss, test_acc = loaded_model.evaluate(X_test,y_test)

"""

200/200 [==============================] - 0s 25us/step


In [35]:
"""

print('test_acc: ',test_acc)

"""

test_acc:  0.675


NameError: name 'X_train' is not defined